# PHE Core

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os

import common_core

## Configuration

Data to download via the API - cases, patients, deaths

In [2]:
PHE_DASHBOARD = "phe-dashboard"

In [3]:
# The 4 nations in the UK
nationNames = [common_core.ENGLAND, common_core.SCOTLAND, common_core.WALES, common_core.NORTHERN_IRELAND]

# The 9 regions in England
regionNames = common_core.regionNames

# The 7 NHS regions in England
nhsRegionNames = common_core.nhsRegionNames

# A selection of lower tier local authorities in Dorset, Hertfordshire, Birmingham, Derbyshire and London (LTLA)
ltlaNames = ["Dorset", "Bournemouth, Christchurch and Poole",
             "Stevenage", "Welwyn Hatfield", "North Hertfordshire", "East Hertfordshire",
             "Sandwell", "Dudley", "Birmingham",
             "Derbyshire Dales", "North East Derbyshire", "High Peak", "Sheffield",
             "Croydon"]

# Combine all of these area types into a single list
areas = [("overview", [common_core.UNITED_KINGDOM]), ("nation", nationNames), ("region", regionNames), ("nhsregion", nhsRegionNames), ("ltla", ltlaNames)]

In [4]:
casesStructure = {
    "cases": "newCasesBySpecimenDate", # Cases by specimen date
    "casesRollingRate": "newCasesBySpecimenDateRollingRate", # Rate of cases per 100K population in 7 day period
    "casesAgeDemographics": "newCasesBySpecimenDateAgeDemographics", # Demographics - 5 year age bands
    "casesReported": "newCasesByPublishDate" # Cases by date reported
}

patientsStructure = {
    "admissions": "newAdmissions", # Patients admitted to hospital
    "admissionsRollingRate": "newAdmissionsRollingRate", # Rate of admissions per 100K population in 7 day period
    "patients": "hospitalCases", # Patients in hospital
    "patientsMv": "covidOccupiedMVBeds", # Patients in mechanical ventilation beds
    "cumAdmissionsByAge": "cumAdmissionsByAge" # Demographics - 05-, 6-17, 18-64, 65-84, 85+
}

deathsStructure = {
    "deaths": "newDeaths28DaysByDeathDate", # Deaths within 28d of +ve test by date of death
    "deathsRollingRate": "newDeaths28DaysByDeathDateRollingRate", # Rate of deaths per 100K population in 7 day period
    "deathsAgeDemographics": "newDeaths28DaysByDeathDateAgeDemographics", # Demographics - 5 year age bands
    "deathsReported": "newDeaths28DaysByPublishDate" # Deaths within 28d of +ve test by date reported
}

onsStructure = {
    "deathsRegistered": "newOnsDeathsByRegistrationDate" # COVID-19 on the death certificate
}

In [5]:
ageDemographics = [
    '00_04', '05_09', '10_14', '15_19', '20_24', '25_29',
    '30_34', '35_39', '40_44', '45_49', '50_54', '55_59',
    '60_64', '65_69', '70_74', '75_79', '80_84', '85_89', '90+'
]

admissionDemographics = [
    '0_to_5', '6_to_17', '18_to_64', '65_to_84', '85+'
]